# SOC Metrics & Executive Dashboard Builder

## Phase
Phase 2 — SOC Visibility Layer

## Objective
Generate threat posture and SOC performance metrics from incident data.


In [1]:
import pandas as pd
from pathlib import Path


In [2]:
PROJECT_ROOT = Path(r"D:\soc-dashboard-suite-main\soc-dashboard-suite-main")

INVESTIGATION_PATH = PROJECT_ROOT / "data" / "enriched" / "soc_investigation_view.csv"

investigation_df = pd.read_csv(INVESTIGATION_PATH, parse_dates=["start_time", "end_time"])
investigation_df.head()


,incident_id,user_id,start_time,end_time,alert_count,incident_severity,investigation_summary,recommended_action
0,INC0001,user1,2025-01-17 00:42:00,2025-01-18 17:13:00,12,high,User user1 triggered 12 alerts involving: auth...,Force password reset and review login history
1,INC0002,user1,2025-01-19 08:26:00,2025-01-20 17:14:00,6,high,User user1 triggered 6 alerts involving: authe...,Force password reset and review login history
2,INC0003,user1,2025-01-21 09:44:00,2025-01-23 08:21:00,5,high,User user1 triggered 5 alerts involving: authe...,Force password reset and review login history
3,INC0004,user1,2025-01-23 09:52:00,2025-01-24 16:07:00,3,medium,User user1 triggered 3 alerts involving: authe...,Monitor account and validate user activity
4,INC0005,user1,2025-01-25 10:53:00,2025-01-27 09:27:00,4,medium,User user1 triggered 4 alerts involving: authe...,Monitor account and validate user activity


In [3]:
incident_volume = investigation_df.groupby("incident_severity").size().reset_index(name="incident_count")
incident_volume


,incident_severity,incident_count
0,high,224
1,low,4
2,medium,75


In [4]:
investigation_df["incident_duration_hours"] = (
    (investigation_df["end_time"] - investigation_df["start_time"]).dt.total_seconds() / 3600
)

duration_stats = investigation_df.groupby("incident_severity")["incident_duration_hours"].mean().reset_index()
duration_stats


,incident_severity,incident_duration_hours
0,high,38.064137
1,low,0.000000
2,medium,21.016000


In [5]:
total_incidents = len(investigation_df)
critical_incidents = len(investigation_df[investigation_df["incident_severity"] == "critical"])

critical_rate = critical_incidents / total_incidents if total_incidents > 0 else 0

print("Critical Incident Rate:", round(critical_rate * 100, 2), "%")


Critical Incident Rate: 0.0 %


In [6]:
affected_users = investigation_df["user_id"].nunique()
print("Unique Users Impacted:", affected_users)


Unique Users Impacted: 50


In [7]:
investigation_df["incident_day"] = investigation_df["start_time"].dt.date

daily_incidents = investigation_df.groupby("incident_day").size().reset_index(name="daily_incident_count")
daily_incidents.head()


,incident_day,daily_incident_count
0,2025-01-17,45
1,2025-01-18,5
2,2025-01-19,35
3,2025-01-20,13
4,2025-01-21,23


In [8]:
METRICS_PATH = PROJECT_ROOT / "data" / "enriched" / "soc_metrics.csv"
TREND_PATH = PROJECT_ROOT / "data" / "enriched" / "incident_trends.csv"

incident_volume.to_csv(METRICS_PATH, index=False)
daily_incidents.to_csv(TREND_PATH, index=False)

print("Saved SOC metrics to:", METRICS_PATH)
print("Saved incident trends to:", TREND_PATH)


Saved SOC metrics to: D:\soc-dashboard-suite-main\soc-dashboard-suite-main\data\enriched\soc_metrics.csv
Saved incident trends to: D:\soc-dashboard-suite-main\soc-dashboard-suite-main\data\enriched\incident_trends.csv
